In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
#print(df_train.shape)
df_train = df_train.drop(columns = ['PassengerId', 'Name','Ticket','Cabin','Embarked'])
df_test = df_test.drop(columns = ['PassengerId', 'Name','Ticket', 'Cabin','Embarked'])
df_train['Sex'] = df_train['Sex'].map({'male' : 0, 'female' : 1})
df_test['Sex'] = df_test['Sex'].map({'male' : 0, 'female' : 1})
df_train.fillna( df_train.mean(), inplace=True)
df_test.fillna( df_test.mean(), inplace=True)
temp = df_train['Survived']
df_train = df_train.drop(columns = ['Survived'])
df_train['Survived'] = temp






In [ ]:
# Separate features and labels
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

# Split the training data into training and validation sets

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=4)
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(df_test)

In [3]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=32)


In [7]:
# Predict probabilities on the test data
submission_df = pd.read_csv('gender_submission.csv')
predictions = model.predict(X_test)


14/14 [==============================] - 0s 1ms/step


In [20]:
type(predictions)
#type(X_test)

numpy.ndarray

In [ ]:
predictions = [predictions[i][0] for i in range(len(predictions))]
predictions = map(round, predictions)
# Convert predictions to DataFrame for better readability
predictions_df = pd.DataFrame({'PassengerId' : submission_df['PassengerId'], 'Survived' : predictions })
res = predictions_df.to_csv('submission.csv', index=False)
submission_df.head(40)